In [ ]:
import time

import numpy as np
import pandas as pd
import sklearn.datasets, sklearn.metrics, sklearn.model_selection, sklearn.tree

import subprocess, sys

In [ ]:
# !cd ../src/ && make print-DEPENDS

In [ ]:
!cd ../src/ && make
subprocess.call([sys.executable, "lltrees_python_import_for_debug.py"])

In [ ]:
X, Y = sklearn.datasets.make_regression(n_samples=1000, n_features=8, n_informative=5, n_targets=1, noise=1, random_state=42)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
sys.path.append('/home/alexandre/Desktop/lltrees/src/build')
import lltrees
conf ={
    'epochs' : 1,
    'learning_rate' : 1,
    'metric_name' : 'mae', # Possible metrics for the moment : mae, mse
    'lltree_max_depth' : 5,
    'lltree_min_size_split' : 2,
    'lltree_criterion' : "absolute_error",  # Possible metrics for the moment : variance, absolute_error
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train, X_test, Y_test)
YP = my_lltree.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

In [ ]:
dtr = sklearn.tree.DecisionTreeRegressor(max_depth = 5, criterion ="absolute_error", random_state = 0)

start_time = time.time()
dtr.fit(X_train,Y_train)
YP = dtr.predict(X_train)
print("--- %s seconds ---" % (time.time() - start_time))

YP = dtr.predict(X_test)
print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

In [ ]:
conf ={
    'epochs' : 50,
    'learning_rate' : 0.1,
    'metric_name' : 'mae', # Possible metrics for the moment : mae, mse
    'lltree_max_depth' : 3,
    'lltree_min_size_split' : 1,
    'lltree_criterion' : "variance",  # Possible metrics for the moment : variance, absolute_error
    'verbose' : 0,  # Possible metrics for the moment : variance, absolute_error
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train, X_test, Y_test)
YP = my_lltree.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
dtr = GradientBoostingRegressor(learning_rate = 0.1, n_estimators =50, 
                                 loss = 'absolute_error', criterion = 'squared_error',
                                 random_state = 0)

start_time = time.time()
dtr.fit(X_train,Y_train)
YP = dtr.predict(X_train)
print("--- %s seconds ---" % (time.time() - start_time))

YP = dtr.predict(X_test)
print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))